In [30]:
import os
import sqlite3
import logging
import time

In [33]:
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'",conn)
tables

,name
0,Customers
1,Products
2,Payments
3,Orders
4,OrderItems


In [42]:
for table in tables['name']:  
    # Looping over each table name stored in the 'tables' DataFrame under column 'name'
    
    print('-'*50, f'{table}', '-'*50)  
    # Prints the table name with separators for visual clarity
    
    print('Count of records:', pd.read_sql(f"select count(*) as count from {table}", conn)['count'].values[0])  
    # Runs a SQL query to count how many rows (records) are present in the current table
    # pd.read_sql(...) executes SQL and returns a DataFrame
    # ['count'] selects the 'count' column, .values[0] fetches the actual number (int)
    
    display(pd.read_sql(f"select * from {table} limit 5", conn))  
    # Fetches and displays the first 5 rows from the current table for preview
    # This gives a glimpse of the structure and data inside the tableindex

-------------------------------------------------- Customers --------------------------------------------------
Count of records: 89316


,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,hCT0x9JiGXBQ,58125,varzea paulista,SP
1,PxA7fv9spyhx,3112,armacao dos buzios,RJ
2,g3nXeJkGI0Qw,4119,jandira,SP
3,EOEsCQ6QlpIg,18212,uberlandia,MG
4,mVz5LO2Vd6cL,88868,ilhabela,SP


-------------------------------------------------- Products --------------------------------------------------
Count of records: 89316


,product_id,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,90K0C1fIyQUf,toys,491.0,19.0,12.0,16.0
1,qejhpMGGVcsl,watches_gifts,440.0,18.0,14.0,17.0
2,qUS5d2pEAyxJ,costruction_tools_garden,2200.0,16.0,16.0,16.0
3,639iGvMyv0De,toys,1450.0,68.0,3.0,48.0
4,1lycYGcsic2F,toys,300.0,17.0,4.0,12.0


-------------------------------------------------- Payments --------------------------------------------------
Count of records: 89316


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,Axfy13Hk4PIk,1,credit_card,1,259.14
1,v6px92oS8cLG,1,credit_card,8,382.39
2,Ulpf9skrhjfm,1,credit_card,4,249.25
3,bwJVWupf2keN,1,credit_card,2,27.79
4,Dd0QnrMk9Cj5,1,credit_card,1,76.15


-------------------------------------------------- Orders --------------------------------------------------
Count of records: 89316


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_timestamp,order_estimated_delivery_date
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,delivered,2017-10-22 18:57:54,2017-10-22 19:14:13,2017-10-26 22:19:52,2017-11-09
1,v6px92oS8cLG,PxA7fv9spyhx,delivered,2018-06-20 21:40:31,2018-06-20 22:20:20,2018-07-03 22:51:22,2018-07-24
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,delivered,2018-02-16 16:19:31,2018-02-17 16:15:35,2018-02-27 01:29:50,2018-03-08
3,bwJVWupf2keN,EOEsCQ6QlpIg,delivered,2018-08-18 18:04:29,2018-08-18 18:15:16,2018-08-27 20:03:51,2018-09-19
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,delivered,2017-12-22 16:44:04,2017-12-22 17:31:31,2018-01-05 19:22:49,2018-01-18


-------------------------------------------------- OrderItems --------------------------------------------------
Count of records: 89316


,order_id,product_id,seller_id,price,shipping_charges
0,Axfy13Hk4PIk,90K0C1fIyQUf,ZWM05J9LcBSF,223.51,84.65
1,v6px92oS8cLG,qejhpMGGVcsl,IjlpYfhUbRQs,170.80,23.79
2,Ulpf9skrhjfm,qUS5d2pEAyxJ,77p2EYxcM9MD,64.40,17.38
3,bwJVWupf2keN,639iGvMyv0De,jWzS0ayv9TGf,264.50,30.72
4,Dd0QnrMk9Cj5,1lycYGcsic2F,l1pYW6GBnPMr,779.90,30.66


In [144]:
orders = pd.read_sql("""SELECT 
    order_id,
    customer_id,
    order_purchase_timestamp,
    order_delivered_timestamp,
    CAST(julianday(order_delivered_timestamp) - julianday(order_purchase_timestamp) AS INTEGER) AS delivery_days
FROM orders 
WHERE order_status = 'delivered'""",conn)
orders

,order_id,customer_id,order_purchase_timestamp,order_delivered_timestamp,delivery_days
0,Axfy13Hk4PIk,hCT0x9JiGXBQ,2017-10-22 18:57:54,2017-10-26 22:19:52,4.0
1,v6px92oS8cLG,PxA7fv9spyhx,2018-06-20 21:40:31,2018-07-03 22:51:22,13.0
2,Ulpf9skrhjfm,g3nXeJkGI0Qw,2018-02-16 16:19:31,2018-02-27 01:29:50,10.0
3,bwJVWupf2keN,EOEsCQ6QlpIg,2018-08-18 18:04:29,2018-08-27 20:03:51,9.0
4,Dd0QnrMk9Cj5,mVz5LO2Vd6cL,2017-12-22 16:44:04,2018-01-05 19:22:49,14.0
...,...,...,...,...,...
87423,zlcLnXGgAabl,bfJtzkBGksdY,2018-08-19 17:25:36,2018-08-30 14:47:46,10.0
87424,IlTAoIkILwrq,4hjsCpj6FC7g,2017-12-15 11:20:27,2017-12-20 19:23:40,5.0
87425,uA6oXfftGVmT,DLn56oKHl9bv,2018-07-24 06:39:14,2018-07-27 01:41:40,2.0
87426,v3QtU6xlGDJp,auSfRYUQUOIq,2018-08-11 08:04:22,2018-08-14 21:43:51,3.0


In [222]:
order_items = pd.read_sql("SELECT order_id,product_id,price,shipping_charges FROM OrderItems ",conn)
order_items[order_items['order_id']=='qJHIiWyTAizV']
o=order_items.groupby(['product_id']).count().reset_index()[['order_id','product_id']]
o
# o['order']

,order_id,product_id
0,1,00BIIFInAdcQ
1,1,00FVdl7l7TLz
2,1,00MeCa7ZMo03
3,1,00V9K0LkMBYL
4,8,00ehq6lzh6MX
...,...,...
27446,4,zzVsv7FKKs9m
27447,2,zzddahYAwwwB
27448,1,zzqhFv79Esms
27449,4,zztEAImyIjb9


In [230]:
payments = pd.read_sql("SELECT order_id,payment_type,payment_value FROM Payments",conn)
payments[payments['order_id']=='303iURxFJEq7']

,order_id,payment_type,payment_value
85441,303iURxFJEq7,wallet,579.93


In [225]:
products = pd.read_sql("SELECT DISTINCT product_id,product_category_name FROM Products WHERE product_weight_g != 'NONE'",conn)
products.value_counts()

product_id    product_category_name
00BIIFInAdcQ  health_beauty            1
fa0jumWDVKJb  toys                     1
fZZkooUyTX8E  toys                     1
fZBBuGxI0ZnA  toys                     1
fZBAFeSVC7mm  toys                     1
                                      ..
KpwP1oTC4NvC  toys                     1
KpdXV7YyopCS  perfumery                1
KpHataK1CGgV  stationery               1
KpDqDpREGaOD  sports_leisure           1
zztLAWHNx6cP  baby                     1
Name: count, Length: 27308, dtype: int64

In [229]:
df = pd.read_sql("""
    SELECT DISTINCT 
        oi.order_id,
        oi.price,
        oi.shipping_charges,
        p.product_id,
        p.product_category_name
    FROM OrderItems oi
    LEFT JOIN Products p 
        ON oi.product_id = p.product_id
    WHERE p.product_weight_g != 'NONE'
""", conn)
df

,order_id,price,shipping_charges,product_id,product_category_name
0,303iURxFJEq7,223.51,61.32,90K0C1fIyQUf,toys
1,Axfy13Hk4PIk,223.51,84.65,90K0C1fIyQUf,toys
2,CsgYlj68ZLq1,223.51,48.35,90K0C1fIyQUf,toys
3,GxVWKhDsAFA0,223.51,27.20,90K0C1fIyQUf,toys
4,GxnIw0TWGbsl,223.51,84.65,90K0C1fIyQUf,toys
...,...,...,...,...,...
89296,COAEBb9wujeO,419.80,50.67,USSzekoHkA25,toys
89297,UcyEqhz0z6is,216.90,25.18,SZRgDV7xa2U8,toys
89298,UxArq2NbNHVg,170.70,41.41,w2OtdEjwNHBr,toys
89299,zlcLnXGgAabl,31.99,188.65,W8vikEizUggJ,toys


In [155]:
customers = pd.read_sql("SELECT customer_id,customer_city,customer_state FROM Customers",conn)
customers

,customer_id,customer_city,customer_state
0,hCT0x9JiGXBQ,varzea paulista,SP
1,PxA7fv9spyhx,armacao dos buzios,RJ
2,g3nXeJkGI0Qw,jandira,SP
3,EOEsCQ6QlpIg,uberlandia,MG
4,mVz5LO2Vd6cL,ilhabela,SP
...,...,...,...
89311,bfJtzkBGksdY,amparo,SP
89312,4hjsCpj6FC7g,itapevi,SP
89313,DLn56oKHl9bv,sao paulo,SP
89314,auSfRYUQUOIq,paulinia,SP


In [265]:
orders_summary = pd.read_sql("""
WITH DeliveredOrders as (
SELECT 
    order_id,
    customer_id,
    DATE(order_purchase_timestamp) AS purchase_date,
    DATE(order_delivered_timestamp) AS delivered_date ,
    CAST(julianday(order_delivered_timestamp) - julianday(order_purchase_timestamp) AS INTEGER) AS delivery_days
FROM orders 
WHERE order_status = 'delivered'),
OrderItem AS (SELECT DISTINCT 
        oi.order_id,
        oi.price,
        oi.shipping_charges,
        p.product_id,
        p.product_category_name
    FROM OrderItems oi
    LEFT JOIN Products p 
        ON oi.product_id = p.product_id
    WHERE p.product_weight_g != 'NONE'),
Payment AS (SELECT order_id,payment_type,payment_value FROM Payments),
Customer AS (SELECT customer_id,customer_city,customer_state FROM Customers) 
SELECT 
o.order_id,
o.customer_id,
o.purchase_date,
o.delivered_date,
o.delivery_days,
oi.product_id,
oi.product_category_name,
oi.price,
oi.shipping_charges,
py.payment_type,
py.payment_value,
c.customer_city,
c.customer_state
FROM DeliveredOrders o 
LEFT JOIN OrderItem oi ON o.order_id = oi.order_id
LEFT JOIN Payment py ON oi.order_id = py.order_id
LEFT JOIN Customer c ON o.customer_id = c.customer_id
""",conn)

In [259]:
orders_summary[orders_summary['order_id']=='v6px92oS8cLG']
orders_summary['customer_id'].nunique()

87428

In [269]:
cursor.execute("DROP TABLE IF EXISTS order_summary;")

cursor.execute("""
CREATE TABLE order_summary(
  OrderId VARCHAR(50) PRIMARY KEY,
  CustomerId VARCHAR(50),
  PurchaseDate DATETIME,
  DeliveredDate DATETIME,
  DeliveryDays INT,
  ProductId VARCHAR(50),
  ProductCategoryName VARCHAR(100),
  Price DECIMAL(15,2),
  Shipping_Charges DECIMAL(15,2),
  Payment_type VARCHAR(50),
  Payment_Value DECIMAL(15,2),
  Customer_City VARCHAR(100),
  Customer_State VARCHAR(50)
);
""")


In [270]:
orders_summary.to_sql('order_summary', conn, if_exists='replace', index=False)

87428

In [248]:
orders_summary['product_category_name'] = orders_summary['product_category_name'].fillna('Unknown')

In [252]:
orders_summary = orders_summary.dropna(subset=['order_delivered_timestamp','product_id', 'price', 'payment_value'])

In [253]:
orders_summary.isnull().sum()

order_id                     0
customer_id                  0
order_purchase_timestamp     0
order_delivered_timestamp    0
delivery_days                0
product_id                   0
product_category_name        0
price                        0
shipping_charges             0
payment_type                 0
payment_value                0
customer_city                0
customer_state               0
dtype: int64

In [266]:
orders_summary.dtypes

order_id                  object
customer_id               object
purchase_date             object
delivered_date            object
delivery_days            float64
product_id                object
product_category_name     object
price                    float64
shipping_charges         float64
payment_type              object
payment_value            float64
customer_city             object
customer_state            object
dtype: object

In [274]:
import sqlite3
import logging
import time 
from ingestion_db import ingest_db

logging.basicConfig(
    filename="logs/get_order_summary.log",
    level = logging.DEBUG,
    format = "%(asctime)s - %(levelname)s - %(message)s",
    filemode="a"
)
def create_order_summary(conn):
    orders_summary = pd.read_sql("""
    WITH DeliveredOrders as (
    SELECT 
    order_id,
    customer_id,
    DATE(order_purchase_timestamp) AS purchase_date,
    DATE(order_delivered_timestamp) AS delivered_date ,
    CEIL(julianday(order_delivered_timestamp) - julianday(order_purchase_timestamp)) AS delivery_days
    FROM orders 
    WHERE order_status = 'delivered'),
    OrderItem AS (SELECT DISTINCT 
        oi.order_id,
        oi.price,
        oi.shipping_charges,
        p.product_id,
        p.product_category_name
    FROM OrderItems oi
    LEFT JOIN Products p 
        ON oi.product_id = p.product_id
    WHERE p.product_weight_g != 'NONE'),
    Payment AS (SELECT order_id,payment_type,payment_value FROM Payments),
    Customer AS (SELECT customer_id,customer_city,customer_state FROM Customers) 
    SELECT 
    o.order_id,
    o.customer_id,
    o.purchase_date,
    o.delivered_date,
    o.delivery_days,
    oi.product_id,
    oi.product_category_name,
    oi.price,
    oi.shipping_charges,
    py.payment_type,
    py.payment_value,
    c.customer_city,
    c.customer_state
    FROM DeliveredOrders o 
    LEFT JOIN OrderItem oi ON o.order_id = oi.order_id
    LEFT JOIN Payment py ON oi.order_id = py.order_id
    LEFT JOIN Customer c ON o.customer_id = c.customer_id
""",conn)
    
    return orders_summary

def clean_data(df):
    '''this function will clean the data'''
    df['product_category_name'] = df['product_category_name'].fillna('Unknown')
    
    df = df.dropna(subset=['delivered_date','product_id', 'price', 'payment_value'])
    
    df['purchase_date'] = pd.to_datetime(df['purchase_date']).dt.date
    df['delivered_date'] = pd.to_datetime(df['delivered_date']).dt.date
    return df

if __name__ == '__main__':
    conn = sqlite3.connect ('inventory.db')
    
    logging.info( 'Creating Order Summary Table.....')
    summary_df = create_order_summary(conn)
    logging.info(summary_df.head())
    
    logging.info('Cleaning Data.....')
    clean_df = clean_data(summary_df)
    logging.info(clean_df.head())
    
    logging.info('Ingesting data.....')
    ingest_db(clean_df, 'order_summary', conn)
    logging.info( 'Completed' )

/var/folders/1w/677cvfp14kd9g4hm22pmcm9c0000gn/T/ipykernel_37741/1470669095.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['purchase_date'] = pd.to_datetime(df['purchase_date']).dt.date
/var/folders/1w/677cvfp14kd9g4hm22pmcm9c0000gn/T/ipykernel_37741/1470669095.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delivered_date'] = pd.to_datetime(df['delivered_date']).dt.date
